In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score as f1

import autosklearn.classification
import warnings
warnings.simplefilter('ignore')

/home/peiwang/anaconda3/lib/python3.7/site-packages/pyparsing.py:3168: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


In [2]:
# run on extracted features

for i in range(6, 9):
    for t in [3600]:
        train_path = './02-feature-extract-ori/features/ft_' + str(i) + '_train.csv'
        test_path = './02-feature-extract-ori/features/ft_' + str(i) + '_test.csv'
        
        df_train = pd.read_csv(train_path)
        df_test = pd.read_csv(test_path)
        
        X_train = np.array(df_train.drop(['label'], axis=1).values.tolist())
        y_train = np.array(df_train['label'].values.tolist())
        X_test = np.array(df_test.drop(['label'], axis=1).values.tolist())
        y_test = np.array(df_test['label'].values.tolist())
        print("data_" + str(i), "training data size:", len(y_train), "test data size:", len(y_test))
        
        
        automl = autosklearn.classification.AutoSklearnClassifier(
            include_estimators=['random_forest'], 
            time_left_for_this_task=3600, 
            initial_configurations_via_metalearning=0, 
            seed = 1,
            ensemble_nbest=1, 
            ensemble_size=1, 
            resampling_strategy='holdout',
            resampling_strategy_arguments={'train_size':0.75, 'random_state':42}
        )
        automl.fit(X_train, y_train, metric=autosklearn.metrics.f1)
        print("\n random-forest-selected, training fscore:", automl.score(X_train, y_train),  
              ", test fscore: ", automl.score(X_test, y_test), '\n')
        print(automl.show_models())
        print(automl.sprint_statistics())
        print('\n')

data_6 training data size: 9167 test data size: 2293
[WARNING] [2020-10-08 14:05:54,560:EnsembleBuilder(1):4ab864c8876c543722442b506ccbf935] No models better than random - using Dummy Score!
[WARNING] [2020-10-08 14:05:54,569:EnsembleBuilder(1):4ab864c8876c543722442b506ccbf935] No models better than random - using Dummy Score!
[WARNING] [2020-10-08 14:27:03,072:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2020-10-08 14:27:03,072:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2020-10-08 14:40:29,255:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2020-10-08 14:40:29,255:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger

 random-forest-selected, training fscore: 0.813124

In [69]:
from sklearn.model_selection import train_test_split
from autosklearn.pipeline.classification import SimpleClassificationPipeline

In [70]:
# analysis for dataset 6:

i = 6

train_path = './02-feature-extract-ori/features/ft_' + str(i) + '_train.csv'
test_path = './02-feature-extract-ori/features/ft_' + str(i) + '_test.csv'
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
        
X_train = np.array(df_train.drop(['label'], axis=1).values.tolist())
y_train = np.array(df_train['label'].values.tolist())
X_test = np.array(df_test.drop(['label'], axis=1).values.tolist())
y_test = np.array(df_test['label'].values.tolist())
print("data_" + str(i), "training data size:", len(y_train), "test data size:", len(y_test))

X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, train_size = 0.75, random_state = 42)
print(len(X_train_train), len(X_valid), len(X_test))

data_6 training data size: 9167 test data size: 2293
6875 2292 2293


In [71]:
# original

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'weighting', 
    'categorical_encoding:__choice__': 'one_hot_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 
    'feature_agglomeration', 
    'rescaling:__choice__': 'none', 
    'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'True', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9950088003474201, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 6, 
    'classifier:random_forest:min_samples_split': 7, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0,
    'classifier:random_forest:n_estimators': 100, 
    'preprocessor:feature_agglomeration:affinity': 'euclidean', 
    'preprocessor:feature_agglomeration:linkage': 'complete', 
    'preprocessor:feature_agglomeration:n_clusters': 91, 
    'preprocessor:feature_agglomeration:pooling_func': 'max', 
    'categorical_encoding:one_hot_encoding:minimum_fraction': 0.0012037695802164811},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('original:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

original:
train: 0.8709677419354839
valid 0.6372007366482505
test 0.6422764227642277


In [72]:
# disable data processing

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 'feature_agglomeration', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9950088003474201, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 6, 
    'classifier:random_forest:min_samples_split': 7, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0,
    'classifier:random_forest:n_estimators': 100, 
    'preprocessor:feature_agglomeration:affinity': 'euclidean', 
    'preprocessor:feature_agglomeration:linkage': 'complete', 
    'preprocessor:feature_agglomeration:n_clusters': 91, 
    'preprocessor:feature_agglomeration:pooling_func': 'max', 
    },
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('data disabled:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

data disabled:
train: 0.8562037797863599
valid 0.6013986013986015
test 0.5801526717557253


In [73]:
# disable feature processing

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 'no_preprocessing', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9950088003474201, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 6, 
    'classifier:random_forest:min_samples_split': 7, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0,
    'classifier:random_forest:n_estimators': 100},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('feature disabled:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

feature disabled:
train: 0.8571428571428572
valid 0.5925925925925926
test 0.5721649484536082


In [75]:
# disable tuned parameter

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 'no_preprocessing', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'gini', #'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.5, #0.9950088003474201, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 1, #6, 
    'classifier:random_forest:min_samples_split': 2, #7, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0,
    'classifier:random_forest:n_estimators': 100},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('parameter disabled:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

parameter disabled:
train: 1.0
valid 0.6198547215496368
test 0.5898123324396781


In [76]:
# analysis for dataset 7:

i = 7

train_path = './02-feature-extract-ori/features/ft_' + str(i) + '_train.csv'
test_path = './02-feature-extract-ori/features/ft_' + str(i) + '_test.csv'
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
        
X_train = np.array(df_train.drop(['label'], axis=1).values.tolist())
y_train = np.array(df_train['label'].values.tolist())
X_test = np.array(df_test.drop(['label'], axis=1).values.tolist())
y_test = np.array(df_test['label'].values.tolist())
print("data_" + str(i), "training data size:", len(y_train), "test data size:", len(y_test))

X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, train_size = 0.75, random_state = 42)
print(len(X_train_train), len(X_valid), len(X_test))

data_7 training data size: 8193 test data size: 2049
6144 2049 2049


In [77]:
# original

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'weighting', 
    'categorical_encoding:__choice__': 'one_hot_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'most_frequent', 
    'preprocessor:__choice__': 'select_percentile_classification', 
    'rescaling:__choice__': 'robust_scaler', 
    'categorical_encoding:one_hot_encoding:use_minimum_fraction': 'True', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'gini', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9008519355763185, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 2, 
    'classifier:random_forest:min_samples_split': 6, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
    'classifier:random_forest:n_estimators': 100, 
    'preprocessor:select_percentile_classification:percentile': 55.84285592896699, 
    'preprocessor:select_percentile_classification:score_func': 'f_classif', 
    'rescaling:robust_scaler:q_max': 0.9194022794180152, 
    'rescaling:robust_scaler:q_min': 0.19454891546620004, 
    'categorical_encoding:one_hot_encoding:minimum_fraction': 0.07845205738658005},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('original:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

original:
train: 0.9769736842105263
valid 0.7739938080495355
test 0.7893333333333333


In [78]:
# data disabled

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'most_frequent', 
    'rescaling:__choice__': 'none',  
    'preprocessor:__choice__': 'select_percentile_classification', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'gini', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9008519355763185, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 2, 
    'classifier:random_forest:min_samples_split': 6, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
    'classifier:random_forest:n_estimators': 100, 
    'preprocessor:select_percentile_classification:percentile': 55.84285592896699, 
    'preprocessor:select_percentile_classification:score_func': 'f_classif'},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('original:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

original:
train: 0.9349955476402494
valid 0.7848101265822784
test 0.7977528089887641


In [79]:
# feature disabled

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'most_frequent', 
    'preprocessor:__choice__': 'no_preprocessing', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'gini', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9008519355763185, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 2, 
    'classifier:random_forest:min_samples_split': 6, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
    'classifier:random_forest:n_estimators': 100},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('original:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

original:
train: 0.940550133096717
valid 0.7834394904458598
test 0.7853107344632768


In [80]:
# parameter disabled

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'most_frequent', 
    'preprocessor:__choice__': 'no_preprocessing', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'gini', #'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.5, #0.9950088003474201, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 1, #6, 
    'classifier:random_forest:min_samples_split': 2, #7, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0,
    'classifier:random_forest:n_estimators': 100},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('original:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

original:
train: 0.9991589571068125
valid 0.7788778877887789
test 0.783625730994152


In [90]:
# analysis for dataset 8:

i = 8

train_path = './02-feature-extract-ori/features/ft_' + str(i) + '_train.csv'
test_path = './02-feature-extract-ori/features/ft_' + str(i) + '_test.csv'
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
        
X_train = np.array(df_train.drop(['label'], axis=1).values.tolist())
y_train = np.array(df_train['label'].values.tolist())
X_test = np.array(df_test.drop(['label'], axis=1).values.tolist())
y_test = np.array(df_test['label'].values.tolist())
print("data_" + str(i), "training data size:", len(y_train), "test data size:", len(y_test))

X_train_train, X_valid, y_train_train, y_valid = train_test_split(X_train, y_train, train_size = 0.75, random_state = 42)
print(len(X_train_train), len(X_valid), len(X_test))

data_8 training data size: 7659 test data size: 1916
5744 1915 1916


In [91]:
# original

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'weighting', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 'feature_agglomeration', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9008524359169225, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 6, 
    'classifier:random_forest:min_samples_split': 4, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
    'classifier:random_forest:n_estimators': 100, 
    'preprocessor:feature_agglomeration:affinity': 'manhattan', 
    'preprocessor:feature_agglomeration:linkage': 'average', 
    'preprocessor:feature_agglomeration:n_clusters': 128, 
    'preprocessor:feature_agglomeration:pooling_func': 'max'},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('original:')

y_train_train_pred = pipe.predict(X_train_train)
  
y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

original:
train: 0.9262048192771085
valid 0.639344262295082
test 0.5621621621621622


In [92]:
# data disabled

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 'feature_agglomeration', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9008524359169225, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 6, 
    'classifier:random_forest:min_samples_split': 4, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
    'classifier:random_forest:n_estimators': 100, 
    'preprocessor:feature_agglomeration:affinity': 'manhattan', 
    'preprocessor:feature_agglomeration:linkage': 'average', 
    'preprocessor:feature_agglomeration:n_clusters': 128, 
    'preprocessor:feature_agglomeration:pooling_func': 'max'},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('data disabled:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

data disabled:
train: 0.8278388278388278
valid 0.5599999999999999
test 0.5562913907284768


In [93]:
# feature disabled

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 'no_preprocessing', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'entropy', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 0.9008524359169225, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 6, 
    'classifier:random_forest:min_samples_split': 4, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
    'classifier:random_forest:n_estimators': 100},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('feature disabled:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

feature disabled:
train: 0.8226691042047533
valid 0.5570469798657718
test 0.5612903225806452


In [97]:
# parameter disabled

pipe = SimpleClassificationPipeline({
    'balancing:strategy': 'none', 
    'categorical_encoding:__choice__': 'no_encoding', 
    'classifier:__choice__': 'random_forest', 
    'imputation:strategy': 'median', 
    'preprocessor:__choice__': 'no_preprocessing', 
    'rescaling:__choice__': 'none', 
    'classifier:random_forest:bootstrap': 'True', 
    'classifier:random_forest:criterion': 'gini', 
    'classifier:random_forest:max_depth': 'None', 
    'classifier:random_forest:max_features': 5, 
    'classifier:random_forest:max_leaf_nodes': 'None', 
    'classifier:random_forest:min_impurity_decrease': 0.0, 
    'classifier:random_forest:min_samples_leaf': 1, 
    'classifier:random_forest:min_samples_split': 2, 
    'classifier:random_forest:min_weight_fraction_leaf': 0.0, 
    'classifier:random_forest:n_estimators': 100},
    dataset_properties={
      'task': 1,
      'sparse': False,
      'multilabel': False,
      'multiclass': False,
      'target_type': 'classification',
      'signed': False}
    )

pipe.fit(X_train_train, y_train_train)

print('parameter disabled:')

y_train_train_pred = pipe.predict(X_train_train)
print('train:', f1(y_train_train, y_train_train_pred))

y_valid_pred = pipe.predict(X_valid)
print('valid', f1(y_valid, y_valid_pred))

y_test_pred = pipe.predict(X_test)
print('test', f1(y_test, y_test_pred))

ValueError: Trying to set illegal value '5' (type '<class 'int'>') for hyperparameter 'classifier:random_forest:max_features, Type: UniformFloat, Range: [0.0, 1.0], Default: 0.5' (default-value has type '<class 'numpy.float64'>').